In [ ]:
import sys
sys.path.insert(0, "..")

In [ ]:
from yolo.models import YOLOv1
from yolo.data.VOC_Dataset import VOC_Dataset
from utils.img_process import show_bbox

# output: 7 * 7 * 30
# S * S * ((x, y, w, h, confidence) * B=2 + C=20)
_voc_root = "/home/ubuntu/VOC2007/VOCdevkit/VOC2007/"
voc_ds = VOC_Dataset(_voc_root)

# bboxes = get_bboxes(bbox_pth)
# for b in bboxes:
#     show_bbox("/home/ubuntu/VOC2007/VOCdevkit/VOC2007/JPEGImages/000048.jpg", b)

In [ ]:
import torch

test = torch.randn(3, 448, 448).unsqueeze(0)
yolo = YOLOv1()
res = yolo(test)